In [35]:
from IPython.display import clear_output
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [36]:
# Initialisation des fichiers de sauvagarde
# Pas besoin de l'executer a chaque fois (sauf si on veut reinitialiser les fichiers)
df = pd.DataFrame({"Photo" : [], "Label" : [], "Separation" : []})
df.to_csv("../Dataset/concat_data_test.csv", index=False, sep=";")
df.to_csv("../Dataset/concat_data_train.csv", index=False, sep=";")
df.to_csv("../Dataset/concat_data_validation.csv", index=False, sep=";")

In [37]:
# Transformation des photos en dataframe
def photo_to_dataframe(chemins_photos: str, label: str, separation: str):
    """Transforme les photos en tableau de pixels et les sauvegarde dans un fichier csv

    Args:
        chemins_photos (str): Chemin des photos (on veut le dossier contenant les photos)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    list_tableau_pixels = []
    list_labels = [label for i in range(len(chemins_photos))]
    list_separation = [separation for i in range(len(chemins_photos))]
    
    print(f"Dossier : {chemins_photos[0][:-10]}") # Afficher le dossier en cours de traitement
    # Parcours et transformation des photos avec barre de progression 
    with tqdm(total=len(chemins_photos), desc="Proccessing" , position=0, leave=True) as pbar:
        for chemin in chemins_photos:
            
            pbar.set_postfix(img=chemin[-10:]) # Mettre à jour la barre de progression avec des informations supplémentaires
            
            img = Image.open(chemin) # 300 x 300
            nouvelle_taille = (50, 50) # 2500 
            img_redimensionnee = img.resize(nouvelle_taille) # Redimensionner l'image
            nb_image = img_redimensionnee.convert('L') # Convertion en noir et blanc
            
            #  Récupération de la liste des pixels
            tab = []
            for i in range(nb_image.size[1]):
                row = []
                for y in range(nb_image.size[0]):
                    # Récupérer la couleur du pixel
                    row.append(nb_image.getpixel((y, i)))
                tab.append(np.array(row))
                
            tab_numpy = np.array(tab) / 255 # mise à l'échelle des données
            list_tableau_pixels.append(tab_numpy) # ajouter le tableau de pixels à la liste finale
            
            pbar.update(1) # Mettre à jour la barre de progression

    clear_output()  # Effacer l'output précédent (sinon pb d'affichage de la barre de progression)
    
    # Création du dataframe
    df2 = pd.DataFrame({"Photo" : list_tableau_pixels, "Label" : list_labels, "Separation" : list_separation})
    
    # Ecriture du dataframe au fur et à mesure
    df2.to_csv("../Dataset/concat_data_" + separation + ".csv", mode='a', header=False, index=False, sep=';')

In [38]:
# Parcours des dossiers et sous-dossiers pour recréer le dataset (recursion)
def recreate_dataset(chemin: str, profondeur: int, label: str, separation: str):
    """Réecriture du dataset: transformation des photos en tableau de pixels et les sauvegarde dans un fichier csv
    Cette fonction est récursive et doit être executer sur un dossier contenant la même arborescence que le dossier ../Dataset

    Args:
        chemin (str): chemin du dataset
        profondeur (int): profondeur de la recursion (elle permet aussi de récupérer les labels et les séparations)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    dossiers = [f for f in os.listdir(chemin) if os.path.isdir(os.path.join(chemin, f))]
    if len(dossiers) == 0: # Si aucun dossier n'est trouvé, on est dans le cas de base
        fichiers_png = [os.path.join(chemin, f) for f in os.listdir(chemin) if f.endswith('.png')] # on recupère et stocke les fichiers .png
        photo_to_dataframe(fichiers_png, label, separation) # On transforme les photos en tableau en df qu'on save dans un fichier.csv
    else : # Sinon, on continue de parcourir les dossiers
        for dossier in dossiers:
            if profondeur == 1:
                separation = dossier
            if profondeur == 2:
                label = dossier
            recreate_dataset(os.path.join(chemin, dossier), profondeur + 1, label, separation)

In [39]:
recreate_dataset("../Dataset", 0, "truc", "bidule") # Appel de la fonction de parcours des dossiers

Dossier : ../Dataset\dataset\train\F\


Proccessing:  51%|█████     | 3291/6464 [00:33<00:31, 99.40it/s, img=003291.png] 


KeyboardInterrupt: 